In [1]:
import io
import re
import numpy as np
import pandas as pd
import numpy.linalg as lin
import warnings 
from gensim.models import Word2Vec
import pickle
from sklearn.cluster import KMeans
warnings.filterwarnings('ignore')

In [2]:
alphabets=['ऄ','अ','आ','इ','ई','उ','ऊ','ऋ','ऌ','ऍ','ऎ','ए','ऐ','ऑ','ऒ','ओ','औ','क','ख','ग','घ','ङ','च','छ','ज','झ','ञ','ट',
'ठ','ड','ढ','ण','त','थ','द','ध','न','ऩ','प','फ','ब','भ','म','य','र','ऱ','ल','ळ','ऴ','व','श','ष','स','ह']

matra=['ऀ','ँ','ं','ः','ऺ','ऻ','़','ा','ि','ी','ु','ू','ृ','ॄ','ॅ','ॆ','े','ै','ॉ','ॊ','ो','ौ','्','ॎ','ॏ','ॕ','ॖ','ॗ']

In [3]:
stopwords=["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप","इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी","इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके","उनको","उन्हीं","उन्हें","उन्हों","उस","उसके","उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर","और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा","का","काफि","काफ़ी","कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर","किस","किसि","किसी","किसे","की","कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ","जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे","जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि","थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे","निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला","बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि","यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग","वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि","सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है","हैं","हो","होता","होति","होती","होते","होना","होने"]
delimiters=["।","?","!"]

In [4]:
sg=Word2Vec.load('sg/hi-d100-m2-sg.model')

In [5]:
file=open('sentence.pkl','rb')
verse_embeddings_sentence=pickle.load(file)
file.close()
file=open('max.pkl','rb')
verse_embeddings_max=pickle.load(file)
file.close()
file=open('mean.pkl','rb')
verse_embeddings_mean=pickle.load(file)
file.close()

In [6]:
file=open('whole.pkl','rb')
verse_embeddings_whole=pickle.load(file)
file.close()

In [7]:
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra+delimiters:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())
def sent_tokenize(text):
    text=remove_special_characters(text)
    regexPattern = '|'.join(map(re.escape, delimiters))
    sentences=re.split(regexPattern,text)
    return [sent.strip() for sent in sentences if len(sent.strip())>0]

In [8]:
def word_embedding(word):
    try:
        return sg.wv[word]
    except:
        return np.array([0.0]*100)

In [9]:
def cosine_similarity(a,b):
    try:
        return a.dot(b)/(lin.norm(a)*lin.norm(b))
    except:
        return -1.0

In [10]:
def ed(a,b):
    return lin.norm(a-b)

In [11]:
def sentence_embedding(sentence):
    sentence=remove_special_characters(sentence)
    words=sentence.split()
    words=[word  for word in words if word not in stopwords]
    if len(words)>0:
        sentence_embedding=[word_embedding(word) for word in words]
        return np.array(list(map(lambda x: sum(x)/len(x), zip(*sentence_embedding))))
    return np.array([0.0]*100)

In [12]:
data=pd.read_csv('../hindi.csv')

In [13]:
query='भौतिक जगत में माया का आधिपत्य होता है और श्रीकृष्ण ने इसे आठवें अध्याय के 15वें श्लोक में अस्थायी और दुखों से भरा हुआ कह कर परिभाषित किया है। इस प्रकार भौतिक शक्ति माया की तुलना अंधकार से की जाती है।'
query_embedding=sentence_embedding(query)
query_embedding_list=query_embedding.reshape(1,-1).astype(np.float64)

# Max Strategy

## Cosine Similarity

In [14]:
def max_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [15]:
verse_index=max_cosine(query_embedding)

In [16]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
309,8,5,अन्तकाले च मामेव स्मरन्मुक्त्वा कलेवरम्।\n\nयः...,अगले श्लोक में श्रीकृष्ण इस सिद्धान्त की व्या...
93,2,52,यदा ते मोहकलिलं बुद्धिर्व्यतितरिष्यति।\n\nतदा ...,श्रीकृष्ण ने पहले के श्लोकों मे स्पष्ट किया है...
519,14,2,इदं ज्ञानमुपाश्रित्य मम साधर्म्यमागताः।सर्गेऽप...,श्रीकृष्ण अर्जुन को आश्वस्त करते हैं कि वे जो ...
112,2,71,विहाय कामान्यः सर्वान्पुमांश्चरति निःस्पृहः।\n...,इस श्लोक में श्रीकृष्ण मनुष्य की शांति के मार...
373,10,7,एतां विभूतिं योगं च मम यो वेत्ति तत्त्वतः।\n\n...,'विभूति' शब्द ब्रह्माण्ड में प्रकट भगवान की प...
638,18,22,यत्तु कृत्स्नवदेकस्मिन्कार्ये सक्तमहैतुकम्।अतत...,जब बुद्धि तमोगुण के प्रभाव से कुंठित हो जाती ह...
333,9,1,श्री भगवानुवाच\n\nइदं तु ते गुह्यतमं प्रवक्ष्य...,इस विषय पर प्रारम्भ में ही श्रीकृष्ण उनके इन उ...
273,6,46,तपस्विभ्योऽधिको योगी ज्ञानिभ्योऽपि मतोऽधिकः।\n...,तपस्वी वह है जो स्वेच्छा से वैराग्य स्वीकार कर...
282,7,8,रसोऽहमप्सु कौन्तेय प्रभास्मि शशिसूर्ययोः।\n\nप...,यह कहने के पश्चात कि वे ही जो सब कुछ दिखाई देत...
420,11,12,दिवि सूर्यसहस्रस्य भवेद्युगपदुत्थिता।\n\nयदि भ...,संजय अब भगवान के विश्वरूपी दिव्य तेज का वर्णन ...


## Euclidean Distance

In [17]:
def max_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_max]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [18]:
verse_index=max_ed(query_embedding)

In [19]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
309,8,5,अन्तकाले च मामेव स्मरन्मुक्त्वा कलेवरम्।\n\nयः...,अगले श्लोक में श्रीकृष्ण इस सिद्धान्त की व्या...
519,14,2,इदं ज्ञानमुपाश्रित्य मम साधर्म्यमागताः।सर्गेऽप...,श्रीकृष्ण अर्जुन को आश्वस्त करते हैं कि वे जो ...
638,18,22,यत्तु कृत्स्नवदेकस्मिन्कार्ये सक्तमहैतुकम्।अतत...,जब बुद्धि तमोगुण के प्रभाव से कुंठित हो जाती ह...
420,11,12,दिवि सूर्यसहस्रस्य भवेद्युगपदुत्थिता।\n\nयदि भ...,संजय अब भगवान के विश्वरूपी दिव्य तेज का वर्णन ...
93,2,52,यदा ते मोहकलिलं बुद्धिर्व्यतितरिष्यति।\n\nतदा ...,श्रीकृष्ण ने पहले के श्लोकों मे स्पष्ट किया है...
112,2,71,विहाय कामान्यः सर्वान्पुमांश्चरति निःस्पृहः।\n...,इस श्लोक में श्रीकृष्ण मनुष्य की शांति के मार...
373,10,7,एतां विभूतिं योगं च मम यो वेत्ति तत्त्वतः।\n\n...,'विभूति' शब्द ब्रह्माण्ड में प्रकट भगवान की प...
207,5,9,प्रलपन्विसृजन्गृह्णन्नुन्मिषन्निमिषन्नपि।\n\nइ...,जब हम किसी महत्वपूर्ण कार्य को कुशलतापूर्वक सम...
206,5,8,नैव किंचित्करोमीति युक्तो मन्येत तत्त्ववित्।\n...,जब हम किसी महत्वपूर्ण कार्य को कुशलतापूर्वक सम...
409,11,1,अर्जुन उवाच\n\nमदनुग्रहाय परमं गुह्यमध्यात्मसं...,अर्जुन भगवान श्रीकृष्ण की विभूतियों को सुनकर औ...


# Mean Pooling

## Cosine Similarity

In [20]:
def mean_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [21]:
verse_index=mean_cosine(query_embedding)

In [22]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
250,6,23,तं विद्याद् दुःखसंयोगवियोगं योगसंज्ञितम्।\n\nस...,भौतिक जगत में माया का आधिपत्य होता है और श्रीक...
324,8,20,परस्तस्मात्तु भावोऽन्योऽव्यक्तोऽव्यक्तात्सनातन...,लौकिक संसारों और उनके अस्थायित्व पर अपना प्रकट...
113,2,72,एषा ब्राह्मी स्थितिः पार्थ नैनां प्राप्य विमुह...,ब्रह्म का अर्थ भगवान तथा स्थिति का अर्थ भगवद्...
336,9,4,मया ततमिदं सर्वं जगदव्यक्तमूर्तिना।\n\nमत्स्था...,वैदिक दर्शन इस अवधारणा को स्वीकार नहीं करता क...
551,15,7,ममैवांशो जीवलोके जीवभूतः सनातनः।मनःषष्ठानीन्द्...,श्रीकृष्ण ने पहले यह समझाया था कि जो उनके लोक ...
320,8,16,आब्रह्मभुवनाल्लोकाः पुनरावर्तिनोऽर्जुन।\n\nमाम...,वैदिक ग्रंथों में पृथ्वी लोक से नीचे सात लोकों...
506,13,24,य एवं वेत्ति पुरुषं प्रकृतिं च गुणैःसह।सर्वथा ...,अज्ञानता आत्मा को दुर्दशा की ओर ले जाती है। भग...
130,3,17,यस्त्वात्मरतिरेव स्यादात्मतृप्तश्च मानवः।\n\nआ...,केवल वे मनुष्य जो बहिर्मुख विषय भोगों की कामना...
313,8,9,कविं पुराणमनुशासितार\n\nमणोरणीयांसमनुस्मरेद्यः...,भगवान का ध्यान विभिन्न प्रकार से किया जा सकता ...
314,8,10,प्रयाणकाले मनसाऽचलेन\n\nभक्त्या युक्तो योगबलेन...,भगवान का ध्यान विभिन्न प्रकार से किया जा सकता ...


## Euclidean Distance

In [23]:
def mean_ed(query_embedding,counter=10):
    scoring=[ed(query_embedding,i) for i in verse_embeddings_mean]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [24]:
verse_index=mean_ed(query_embedding)

In [25]:
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
250,6,23,तं विद्याद् दुःखसंयोगवियोगं योगसंज्ञितम्।\n\nस...,भौतिक जगत में माया का आधिपत्य होता है और श्रीक...
336,9,4,मया ततमिदं सर्वं जगदव्यक्तमूर्तिना।\n\nमत्स्था...,वैदिक दर्शन इस अवधारणा को स्वीकार नहीं करता क...
551,15,7,ममैवांशो जीवलोके जीवभूतः सनातनः।मनःषष्ठानीन्द्...,श्रीकृष्ण ने पहले यह समझाया था कि जो उनके लोक ...
113,2,72,एषा ब्राह्मी स्थितिः पार्थ नैनां प्राप्य विमुह...,ब्रह्म का अर्थ भगवान तथा स्थिति का अर्थ भगवद्...
320,8,16,आब्रह्मभुवनाल्लोकाः पुनरावर्तिनोऽर्जुन।\n\nमाम...,वैदिक ग्रंथों में पृथ्वी लोक से नीचे सात लोकों...
313,8,9,कविं पुराणमनुशासितार\n\nमणोरणीयांसमनुस्मरेद्यः...,भगवान का ध्यान विभिन्न प्रकार से किया जा सकता ...
314,8,10,प्रयाणकाले मनसाऽचलेन\n\nभक्त्या युक्तो योगबलेन...,भगवान का ध्यान विभिन्न प्रकार से किया जा सकता ...
287,7,13,त्रिभिर्गुणमयैर्भावैरेभिः सर्वमिदं जगत्।\n\nमो...,पिछले श्लोकों को सुनकर अर्जुन यह सोचता होगा “ह...
130,3,17,यस्त्वात्मरतिरेव स्यादात्मतृप्तश्च मानवः।\n\nआ...,केवल वे मनुष्य जो बहिर्मुख विषय भोगों की कामना...
506,13,24,य एवं वेत्ति पुरुषं प्रकृतिं च गुणैःसह।सर्वथा ...,अज्ञानता आत्मा को दुर्दशा की ओर ले जाती है। भग...


# Line Matching

## Cosine Similarity

In [26]:
def line_cosine(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[cosine_similarity(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [27]:
verse_index=line_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
250,6,23,तं विद्याद् दुःखसंयोगवियोगं योगसंज्ञितम्।\n\nस...,भौतिक जगत में माया का आधिपत्य होता है और श्रीक...
339,9,7,सर्वभूतानि कौन्तेय प्रकृतिं यान्ति मामिकाम्।\n...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
340,9,8,प्रकृतिं स्वामवष्टभ्य विसृजामि पुनः पुनः।\n\nभ...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
678,18,62,तमेव शरणं गच्छ सर्वभावेन भारत।तत्प्रसादात्परां...,भगवान पर निर्भरता के कारण आत्मा को अपनी वर्तमा...
279,7,5,अपरेयमितस्त्वन्यां प्रकृतिं विद्धि मे पराम्।\n...,श्रीकृष्ण अब पूर्णतया भौतिक ज्ञान के क्षेत्र स...
408,10,42,अथवा बहुनैतेन किं ज्ञातेन तवार्जुन।\n\nविष्टभ्...,श्रीकृष्ण का उपर्युक्त कथन यह इंगित करता है कि...
129,3,16,एवं प्रवर्तितं चक्रं नानुवर्तयतीह यः।\n\nअघायु...,चक्र या चरण से तात्पर्य घटनाओं की क्रमबद्धता स...
333,9,1,श्री भगवानुवाच\n\nइदं तु ते गुह्यतमं प्रवक्ष्य...,इस विषय पर प्रारम्भ में ही श्रीकृष्ण उनके इन उ...
356,9,24,अहं हि सर्वयज्ञानां भोक्ता च प्रभुरेव च।\n\nन ...,श्रीकृष्ण अब देवताओं की पूजा करने के दोषों को...
416,11,8,न तु मां शक्यसे द्रष्टुमनेनैव स्वचक्षुषा।\n\nद...,जब परमप्रभु संसार में अवतरित होते हैं तब उनके ...


## Euclidean Distance

In [28]:
def line_ed(query_embeddings,counter=10):
    scoring=[]
    for embeddings in verse_embeddings_sentence:
        score=[ed(query_embedding,embed) for embed in embeddings]
        scoring.append(max(score))
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [29]:
verse_index=line_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
309,8,5,अन्तकाले च मामेव स्मरन्मुक्त्वा कलेवरम्।\n\nयः...,अगले श्लोक में श्रीकृष्ण इस सिद्धान्त की व्या...
430,11,22,रुद्रादित्या वसवो ये च साध्या\n\nविश्वेऽश्िवनौ...,ये सभी महान विभूतियाँ भगवान की शक्ति से ही अप...
457,11,49,मा ते व्यथा मा च विमूढभावो\n\nदृष्ट्वा रूपं घो...,श्रीकृष्ण निरन्तर अर्जुन को धीरज बंधाते हुए कह...
409,11,1,अर्जुन उवाच\n\nमदनुग्रहाय परमं गुह्यमध्यात्मसं...,अर्जुन भगवान श्रीकृष्ण की विभूतियों को सुनकर औ...
613,17,25,तदित्यनभिसन्धाय फलं यज्ञतपःक्रियाः।\n\nदानक्रि...,हमारे द्वारा किए जाने वाले कर्मों का फल देना भ...
272,6,45,प्रयत्नाद्यतमानस्तु योगी संशुद्धकिल्बिषः।\n\nअ...,पिछले जन्म के संचित अभ्यास आध्यात्मिक उन्नति क...
160,4,4,अर्जुन उवाच\n\nअपरं भवतो जन्म परं जन्म विवस्वत...,अर्जुन भगवान श्रीकृष्ण के कथनों में घटनाओं की ...
654,18,38,विषयेन्द्रियसंयोगाद्यत्तदग्रेऽमृतोपमम्।परिणामे...,राजसिक सुख एक रोमांच की भांति है जो इन्द्रियों...
62,2,21,वेदाविनाशिनं नित्यं य एनमजमव्ययम्।\n\nकथं स पु...,आध्यात्मिक रूप से उन्नत आत्मा हमारे उस अहं को ...
607,17,19,मूढग्राहेणात्मनो यत्पीडया क्रियते तपः।परस्योत्...,मूढग्राहेणात्मनो शब्द ऐसे व्यक्तियों के लिए प्...


# Whole Part

## Cosine Similarity

In [30]:
def whole_cosine(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[-1::-1][:counter]
    return verse_index

In [31]:
verse_index=whole_cosine(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
250,6,23,तं विद्याद् दुःखसंयोगवियोगं योगसंज्ञितम्।\n\nस...,भौतिक जगत में माया का आधिपत्य होता है और श्रीक...
324,8,20,परस्तस्मात्तु भावोऽन्योऽव्यक्तोऽव्यक्तात्सनातन...,लौकिक संसारों और उनके अस्थायित्व पर अपना प्रकट...
336,9,4,मया ततमिदं सर्वं जगदव्यक्तमूर्तिना।\n\nमत्स्था...,वैदिक दर्शन इस अवधारणा को स्वीकार नहीं करता क...
113,2,72,एषा ब्राह्मी स्थितिः पार्थ नैनां प्राप्य विमुह...,ब्रह्म का अर्थ भगवान तथा स्थिति का अर्थ भगवद्...
551,15,7,ममैवांशो जीवलोके जीवभूतः सनातनः।मनःषष्ठानीन्द्...,श्रीकृष्ण ने पहले यह समझाया था कि जो उनके लोक ...
340,9,8,प्रकृतिं स्वामवष्टभ्य विसृजामि पुनः पुनः।\n\nभ...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
339,9,7,सर्वभूतानि कौन्तेय प्रकृतिं यान्ति मामिकाम्।\n...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
322,8,18,अव्यक्ताद्व्यक्तयः सर्वाः प्रभवन्त्यहरागमे।\n\...,ब्रह्माण्ड के अद्भुत ब्रह्माण्डीय खेल में विभि...
338,9,6,यथाऽऽकाशस्थितो नित्यं वायुः सर्वत्रगो महान्।\n...,श्रीकृष्ण ने चौथे श्लोक से छठे श्लोक तक 'मत्स्...
320,8,16,आब्रह्मभुवनाल्लोकाः पुनरावर्तिनोऽर्जुन।\n\nमाम...,वैदिक ग्रंथों में पृथ्वी लोक से नीचे सात लोकों...


## Euclidean Distance

In [32]:
def whole_ed(query_embedding,counter=10):
    scoring=[cosine_similarity(query_embedding,i) for i in verse_embeddings_whole]
    verse_index=np.argsort(scoring)[:counter]
    return verse_index

In [33]:
verse_index=whole_ed(query_embedding)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
22,1,26,तत्रापश्यत्स्थितान्पार्थः पितृ़नथ पितामहान्।\n...,अर्जुन युद्धभूमि में अपने सभी सम्बंधियों को दे...
486,13,4,तत्क्षेत्रं यच्च यादृक् च यद्विकारि यतश्च यत्।...,श्रीकृष्ण अब स्वयं कई प्रश्न खड़े करते हैं और ...
597,17,9,कट्वम्ललवणात्युष्णतीक्ष्णरूक्षविदाहिनः।आहारा र...,"जब शाकाहारी भोजनों को अत्यधिक मीर्च, शर्करा, न..."
10,1,13,ततः शङ्खाश्च भेर्यश्च पणवानकगोमुखाः।\n\nसहसैवा...,युद्ध के लिए भीष्म पितामह के तीव्र उत्साह को द...
598,17,10,यातयामं गतरसं पूति पर्युषितं च यत्।उच्छिष्टमपि...,ऐसे पके हुए भोज्य पदार्थ जिन्हें पकाये हुए एक ...
8,1,11,अयनेषु च सर्वेषु यथाभागमवस्थिताः।\n\nभीष्ममेवा...,दुर्योधन यह समझ गया था कि भीष्म पितामह मानसिक ...
3,1,4,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि।\n\nयुयु...,अपने सम्मख संकट को मंडराते देखकर दुर्योधन को प...
4,1,5,धृष्टकेतुश्चेकितानः काशिराजश्च वीर्यवान्।\n\nप...,अपने सम्मख संकट को मंडराते देखकर दुर्योधन को प...
5,1,6,युधामन्युश्च विक्रान्त उत्तमौजाश्च वीर्यवान्।\...,अपने सम्मख संकट को मंडराते देखकर दुर्योधन को प...
2,1,3,पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम्।\n\...,दुर्योधन एक कुशल कूटनीतिज्ञ के रूप में अपने से...


# Clustering

## Max Pooling

In [34]:
kmeans_max = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_max)

In [35]:
def kmeans_max_cluster(query_embedding_list):
    label=kmeans_max.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_max.labels_[i]==label]
    return verse_index

In [36]:
verse_index=kmeans_max_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
10,1,13,ततः शङ्खाश्च भेर्यश्च पणवानकगोमुखाः।\n\nसहसैवा...,युद्ध के लिए भीष्म पितामह के तीव्र उत्साह को द...
24,1,28,अर्जुन उवाच\n\nकृपया परयाऽऽविष्टो विषीदन्निदमब...,आसक्ति लौकिक या आध्यात्मिक मनोभावना हो सकती है...
62,2,21,वेदाविनाशिनं नित्यं य एनमजमव्ययम्।\n\nकथं स पु...,आध्यात्मिक रूप से उन्नत आत्मा हमारे उस अहं को ...
64,2,23,नैनं छिन्दन्ति शस्त्राणि नैनं दहति पावकः।\n\nन...,"चेतना, जो आत्मा का लक्षण है, उसे भौतिक अवयवों..."
69,2,28,अव्यक्तादीनि भूतानि व्यक्तमध्यानि भारत।\n\nअव्...,इस अध्याय के श्लोक 2.20 में श्रीकृष्ण ने आत्मा...
197,4,41,योगसंन्यस्तकर्माणं ज्ञानसंछिन्नसंशयम्।\n\nआत्म...,'कर्म' में निश्चित धार्मिक विधि-विधान और सामाज...
198,4,42,तस्मादज्ञानसंभूतं हृत्स्थं ज्ञानासिनाऽऽत्मनः।\...,यहाँ हृदय शब्द का अर्थ वक्षस्थल में स्थित शारी...
218,5,20,न प्रहृष्येत्प्रियं प्राप्य नोद्विजेत्प्राप्य ...,इस श्लोक का यह अंश 'न तो सुख में प्रसन्न होना ...
280,7,6,एतद्योनीनि भूतानि सर्वाणीत्युपधारय।\n\nअहं कृत...,भौतिक जगत के सभी प्राणी आत्मा और पदार्थ के संय...
316,8,12,सर्वद्वाराणि संयम्य मनो हृदि निरुध्य च।\n\nमूर...,शारीरिक इन्द्रियों के माध्यम से मन संसार में ...


## Mean Pooling

In [37]:
kmeans_mean = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_mean)

In [38]:
def kmeans_mean_cluster(query_embedding_list):
    label=kmeans_mean.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_mean.labels_[i]==label]
    return verse_index

In [39]:
verse_index=kmeans_mean_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
37,1,41,अधर्माभिभवात्कृष्ण प्रदुष्यन्ति कुलस्त्रियः।\n...,वैदिक सभ्यता में समाज में स्त्रियों को उच्च स...
81,2,40,नेहाभिक्रमनाशोऽस्ति प्रत्यवायो न विद्यते।\n\nस...,हमारे सामने सबसे बड़ा भय यह है कि अगले जन्म मे...
83,2,42,यामिमां पुष्पितां वाचं प्रवदन्त्यविपश्चितः।\n\...,वेदों को तीन खण्डो में विभक्त किया गया है। ये ...
84,2,43,कामात्मानः स्वर्गपरा जन्मकर्मफलप्रदाम्।\n\nक्र...,वेदों को तीन खण्डो में विभक्त किया गया है। ये ...
96,2,55,श्री भगवानुवाच\n\nप्रजहाति यदा कामान् सर्वान् ...,इस श्लोक से श्रीकृष्ण अर्जुन के प्रश्नों का उ...
124,3,11,देवान्भावयतानेन ते देवा भावयन्तु वः।\n\nपरस्पर...,स्वर्ग के देवता अधिकारिक रूप से ब्रह्मांड के प...
130,3,17,यस्त्वात्मरतिरेव स्यादात्मतृप्तश्च मानवः।\n\nआ...,केवल वे मनुष्य जो बहिर्मुख विषय भोगों की कामना...
192,4,36,अपि चेदसि पापेभ्यः सर्वेभ्यः पापकृत्तमः।\n\nसर...,मायाबद्ध संसार गहरे समुद्र की भांति है जिसमें ...
193,4,37,यथैधांसि समिद्धोऽग्निर्भस्मसात्कुरुतेऽर्जुन।\n...,अग्नि की छोटी-सी चिंगारी भी भयंकर आग का रूप धा...
217,5,19,इहैव तैर्जितः सर्गो येषां साम्ये स्थितं मनः।\n...,श्रीकृष्ण ने यहाँ 'सम' शब्द का प्रयोग किया है...


## Whole

In [40]:
kmeans_whole = KMeans(n_clusters=60, random_state=0).fit(verse_embeddings_whole)

In [41]:
def kmeans_whole_cluster(query_embedding_list):
    label=kmeans_whole.predict(query_embedding_list)[0]
    verse_index=[i for i in range(len(data)) if kmeans_whole.labels_[i]==label]
    return verse_index

In [42]:
verse_index=kmeans_whole_cluster(query_embedding_list)
data.loc[verse_index]

,Chapter,Verse,Sloke,Commentary
280,7,6,एतद्योनीनि भूतानि सर्वाणीत्युपधारय।\n\nअहं कृत...,भौतिक जगत के सभी प्राणी आत्मा और पदार्थ के संय...
308,8,4,अधिभूतं क्षरो भावः पुरुषश्चाधिदैवतम्।\n\nअधियज...,"पाँच तत्त्वों-पृथ्वी, जल, अग्नि, वायु और आकाश..."
321,8,17,सहस्रयुगपर्यन्तमहर्यद्ब्रह्मणो विदुः।\n\nरात्र...,वैदिक ब्रह्माण्डीय ज्ञान पद्धति में समय की गणन...
322,8,18,अव्यक्ताद्व्यक्तयः सर्वाः प्रभवन्त्यहरागमे।\n\...,ब्रह्माण्ड के अद्भुत ब्रह्माण्डीय खेल में विभि...
323,8,19,भूतग्रामः स एवायं भूत्वा भूत्वा प्रलीयते।\n\nर...,वेदों में चार प्रकार की प्रलय का उल्लेख किया ग...
324,8,20,परस्तस्मात्तु भावोऽन्योऽव्यक्तोऽव्यक्तात्सनातन...,लौकिक संसारों और उनके अस्थायित्व पर अपना प्रकट...
339,9,7,सर्वभूतानि कौन्तेय प्रकृतिं यान्ति मामिकाम्।\n...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
340,9,8,प्रकृतिं स्वामवष्टभ्य विसृजामि पुनः पुनः।\n\nभ...,पिछले दो श्लोकों में श्रीकृष्ण ने यह व्याख्या ...
502,13,20,प्रकृतिं पुरुषं चैव विद्ध्यनादी उभावपि।विकारां...,भौतिक शक्ति को माया या प्रकृति कहा जाता है। भग...
520,14,3,मम योनिर्महद्ब्रह्म तस्मिन् गर्भं दधाम्यहम्।सं...,जैसा कि 7वें और 8वें अध्याय में वर्णन किया गया...
